In [3]:
import re
from autocorrect import Speller
import pandas as pd
import psycopg2
import datetime as dt
import numpy as np
import seaborn as sns
 
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

In [4]:
energy_raw = pd.read_csv("../data/raw/Electric_Consumption_And_Cost__2010_-_Feb_2023__20240417.csv")
energy_raw.head(5)

,Development Name,Borough,Account Name,Location,Meter AMR,Meter Scope,TDS #,EDP,RC Code,Funding Source,...,Meter Number,Estimated,Current Charges,Rate Class,Bill Analyzed,Consumption (KWH),KWH Charges,Consumption (KW),KW Charges,Other charges
0,ADAMS,BRONX,ADAMS,BLD 05,NONE,BLD 01 to 07,118.0,248,B011800,FEDERAL,...,7223256,N,15396.82,GOV/NYC/068,Yes,128800.0,7387.97,216.0,2808.0,5200.85
1,ADAMS,BRONX,ADAMS,BLD 05,NONE,BLD 01 to 07,118.0,248,B011800,FEDERAL,...,7223256,N,14556.34,GOV/NYC/068,Yes,115200.0,6607.87,224.0,2912.0,5036.47
2,ADAMS,BRONX,ADAMS,BLD 05,NONE,BLD 01 to 07,118.0,248,B011800,FEDERAL,...,7223256,N,13904.98,GOV/NYC/068,Yes,103200.0,5919.55,216.0,2808.0,5177.43
3,ADAMS,BRONX,ADAMS,BLD 05,NONE,BLD 01 to 07,118.0,248,B011800,FEDERAL,...,7223256,N,14764.04,GOV/NYC/068,Yes,105600.0,6057.22,208.0,2704.0,6002.82
4,ADAMS,BRONX,ADAMS,BLD 05,NONE,BLD 01 to 07,118.0,248,B011800,FEDERAL,...,7223256,N,13729.54,GOV/NYC/068,Yes,97600.0,5598.34,216.0,2808.0,5323.20


In [5]:
energy_raw.columns

Index(['Development Name', 'Borough', 'Account Name', 'Location', 'Meter AMR',
       'Meter Scope', 'TDS #', 'EDP', 'RC Code', 'Funding Source', 'AMP #',
       'Vendor Name', 'UMIS BILL ID', 'Revenue Month', 'Service Start Date',
       'Service End Date', '# days', 'Meter Number', 'Estimated',
       'Current Charges', 'Rate Class', 'Bill Analyzed', 'Consumption (KWH)',
       'KWH Charges', 'Consumption (KW)', 'KW Charges', 'Other charges'],
      dtype='object')

In [6]:
energy_raw.shape

(447849, 27)

## Data Cleaning Tasks 

#### Task 1: Identify and Remove Irrelevant Features

In [7]:
energy_selected_column =energy_raw[["Borough", "Account Name","Meter Number","Funding Source","Current Charges",\
                                     'Consumption (KWH)','KWH Charges', 'Consumption (KW)', 'KW Charges',\
                                     "Revenue Month","Service Start Date", "Service End Date"]]
energy_selected_column.head()

,Borough,Account Name,Meter Number,Funding Source,Current Charges,Consumption (KWH),KWH Charges,Consumption (KW),KW Charges,Revenue Month,Service Start Date,Service End Date
0,BRONX,ADAMS,7223256,FEDERAL,15396.82,128800.0,7387.97,216.0,2808.0,2010-01,12/24/09,1/26/10
1,BRONX,ADAMS,7223256,FEDERAL,14556.34,115200.0,6607.87,224.0,2912.0,2010-02,1/26/10,2/25/10
2,BRONX,ADAMS,7223256,FEDERAL,13904.98,103200.0,5919.55,216.0,2808.0,2010-03,2/25/10,3/26/10
3,BRONX,ADAMS,7223256,FEDERAL,14764.04,105600.0,6057.22,208.0,2704.0,2010-04,3/26/10,4/26/10
4,BRONX,ADAMS,7223256,FEDERAL,13729.54,97600.0,5598.34,216.0,2808.0,2010-05,4/26/10,5/24/10


### Task 1: Remove columns

In [8]:
energy_selected_column.rename(columns= 
                                {"Borough":"borough", 
                                "Account Name" :"account_name",
                                "Meter Number":"serial_number",
                                "Funding Source":"funding_origin",
                                "Current Charges":"total_bill",
                                "Revenue Month":"year_month",
                                "Consumption (KWH)": "kwh_consumption", # kwh_usage
                                "KWH Charges": "kwh_bill",
                                "Consumption (KW)":"kw_consumption",
                                "KW Charges": "kw_bill",
                                "Service Start Date":"start_date",
                                "Service End Date":"end_date"},inplace = True)

energy_selected_column.head(2)

,borough,account_name,serial_number,funding_origin,total_bill,kwh_consumption,kwh_bill,kw_consumption,kw_bill,year_month,start_date,end_date
0,BRONX,ADAMS,7223256,FEDERAL,15396.82,128800.0,7387.97,216.0,2808.0,2010-01,12/24/09,1/26/10
1,BRONX,ADAMS,7223256,FEDERAL,14556.34,115200.0,6607.87,224.0,2912.0,2010-02,1/26/10,2/25/10


### Task 2: Handle Missing Data

In [9]:
energy_handle_missing_data = energy_selected_column.copy()
energy_handle_missing_data.isnull().sum().sum()

16

In [10]:
energy_handle_missing_data.isnull().sum()

borough            0
account_name       0
serial_number      0
funding_origin     0
total_bill         0
kwh_consumption    0
kwh_bill           0
kw_consumption     0
kw_bill            0
year_month         0
start_date         8
end_date           8
dtype: int64

In [11]:
energy_handle_missing_data[energy_handle_missing_data.start_date.isnull()]

,borough,account_name,serial_number,funding_origin,total_bill,kwh_consumption,kwh_bill,kw_consumption,kw_bill,year_month,start_date,end_date
294254,MANHATTAN,LOWER EAST SIDE II,5503302,FEDERAL,0.00,0.0,0.0,0.0,0.0,2020-04,NaN,NaN
297117,BROOKLYN,RED HOOK EAST,9956008,FEDERAL,15.96,0.0,0.0,0.0,0.0,2020-02,NaN,NaN
297120,BROOKLYN,RED HOOK EAST,9956007,FEDERAL,20.22,0.0,0.0,0.0,0.0,2020-02,NaN,NaN
337554,MANHATTAN,LOWER EAST SIDE II,5503302,FEDERAL,0.00,0.0,0.0,0.0,0.0,2020-04,NaN,NaN
337555,MANHATTAN,LOWER EAST SIDE II,5503302,FEDERAL,0.04,0.0,0.0,0.0,0.0,2020-06,NaN,NaN
337556,MANHATTAN,LOWER EAST SIDE II,5503302,FEDERAL,0.00,0.0,0.0,0.0,0.0,2020-05,NaN,NaN
347588,BROOKLYN,RED HOOK EAST,9956008,FEDERAL,15.96,0.0,0.0,0.0,0.0,2020-02,NaN,NaN
347592,BROOKLYN,RED HOOK EAST,9956007,FEDERAL,20.22,0.0,0.0,0.0,0.0,2020-02,NaN,NaN


### Task 2: Handle Missing Data

In [12]:
columns_to_check= ["start_date","end_date"]
energy_dropna_columns = energy_handle_missing_data.dropna(subset= columns_to_check)

In [13]:
energy_dropna_columns.info()

<class 'pandas.core.frame.DataFrame'>
Index: 447841 entries, 0 to 447848
Data columns (total 12 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   borough          447841 non-null  object 
 1   account_name     447841 non-null  object 
 2   serial_number    447841 non-null  object 
 3   funding_origin   447841 non-null  object 
 4   total_bill       447841 non-null  float64
 5   kwh_consumption  447841 non-null  float64
 6   kwh_bill         447841 non-null  float64
 7   kw_consumption   447841 non-null  float64
 8   kw_bill          447841 non-null  float64
 9   year_month       447841 non-null  object 
 10  start_date       447841 non-null  object 
 11  end_date         447841 non-null  object 
dtypes: float64(5), object(7)
memory usage: 44.4+ MB


### Task 3: fix the data type

In [23]:
energy_dropna_columns['start_date'] = pd.to_datetime(energy_dropna_columns['start_date'])
energy_dropna_columns['end_date'] = pd.to_datetime(energy_dropna_columns['end_date'])


energy_dropna_columns['year_month'] = energy_dropna_columns['year_month'].astype(str) + '-01'
energy_dropna_columns['year_month'] = pd.to_datetime(energy_dropna_columns['year_month'])
#energy_selected_column['revenue_month']  = energy_selected_column['revenue_month'].dt.strftime('%Y-%m-%d')


energy_dropna_columns['total_bill'] = energy_dropna_columns['total_bill'].astype(float)
energy_dropna_columns['kwh_consumption'] = energy_dropna_columns ['kwh_consumption'].astype(float)
energy_dropna_columns['kw_consumption'] = energy_dropna_columns ['kw_consumption'].astype(float)
energy_dropna_columns['kwh_consumption'] = energy_dropna_columns ['kwh_consumption'].astype(float)
energy_dropna_columns['kwh_bill'] = energy_dropna_columns ['kw_bill'].astype(float)
energy_dropna_columns.dtypes


borough                    object
account_name               object
serial_number              object
funding_origin             object
total_bill                float64
kwh_consumption           float64
kwh_bill                  float64
kw_consumption            float64
kw_bill                   float64
year_month         datetime64[ns]
start_date         datetime64[ns]
end_date           datetime64[ns]
dtype: object

In [15]:
energy_dropna_columns.head()

,borough,account_name,serial_number,funding_origin,total_bill,kwh_consumption,kwh_bill,kw_consumption,kw_bill,year_month,start_date,end_date
0,BRONX,ADAMS,7223256,FEDERAL,15396.82,128800.0,2808.0,216.0,2808.0,2010-01-01,2009-12-24,2010-01-26
1,BRONX,ADAMS,7223256,FEDERAL,14556.34,115200.0,2912.0,224.0,2912.0,2010-02-01,2010-01-26,2010-02-25
2,BRONX,ADAMS,7223256,FEDERAL,13904.98,103200.0,2808.0,216.0,2808.0,2010-03-01,2010-02-25,2010-03-26
3,BRONX,ADAMS,7223256,FEDERAL,14764.04,105600.0,2704.0,208.0,2704.0,2010-04-01,2010-03-26,2010-04-26
4,BRONX,ADAMS,7223256,FEDERAL,13729.54,97600.0,2808.0,216.0,2808.0,2010-05-01,2010-04-26,2010-05-24


# Task 4: Clean Textual Data
- If applicable, clean textual data by removing extra spaces, correcting typos
- standardizing text (e.g., converting to lowercase).

In [16]:
energy_clean_textual_data = energy_dropna_columns.copy()
energy_clean_textual_data.head(2)

,borough,account_name,serial_number,funding_origin,total_bill,kwh_consumption,kwh_bill,kw_consumption,kw_bill,year_month,start_date,end_date
0,BRONX,ADAMS,7223256,FEDERAL,15396.82,128800.0,2808.0,216.0,2808.0,2010-01-01,2009-12-24,2010-01-26
1,BRONX,ADAMS,7223256,FEDERAL,14556.34,115200.0,2912.0,224.0,2912.0,2010-02-01,2010-01-26,2010-02-25


In [17]:
# Convert to lowercase
energy_clean_textual_data['borough'] = energy_clean_textual_data['borough'].str.lower()
energy_clean_textual_data['account_name'] = energy_clean_textual_data['account_name'].str.lower()
energy_clean_textual_data['funding_origin'] = energy_clean_textual_data['funding_origin'].str.lower()

In [24]:
# Remove extra spaces
energy_clean_textual_data['borough'] = energy_clean_textual_data['borough'].str.strip()  
energy_clean_textual_data['account_name'] = energy_clean_textual_data['account_name'].str.strip()  
energy_clean_textual_data['funding_origin'] = energy_clean_textual_data['funding_origin'].str.strip() 

# Replace multiple spaces with a single space
energy_clean_textual_data['borough'] = energy_clean_textual_data['borough'].str.replace(r'\s+', ' ', regex=True)
energy_clean_textual_data['account_name'] = energy_clean_textual_data['account_name'].str.replace(r'\s+', ' ', regex=True)

# Correct typos
# spell = Speller(lang='en')
# energy_clean_textual_data['borough'] = energy_clean_textual_data['borough'].apply(lambda x: spell(x))

# Remove punctuation
energy_clean_textual_data['borough'] = energy_clean_textual_data['borough'].str.replace(r'[^\w\s]', '', regex=True)
energy_clean_textual_data['account_name'] = energy_clean_textual_data['account_name'].str.replace(r'[^\w\s]', '', regex=True)

### Task 6: Remove Duplicates


In [19]:
energy_remove_duplicate = energy_clean_textual_data.copy()
energy_remove_duplicate.head(2)

,borough,account_name,serial_number,funding_origin,total_bill,kwh_consumption,kwh_bill,kw_consumption,kw_bill,year_month,start_date,end_date
0,bronx,adams,7223256,federal,15396.82,128800.0,2808.0,216.0,2808.0,2010-01-01,2009-12-24,2010-01-26
1,bronx,adams,7223256,federal,14556.34,115200.0,2912.0,224.0,2912.0,2010-02-01,2010-01-26,2010-02-25


In [25]:
# Identify duplicates

# dentify duplicates
duplicates = energy_remove_duplicate.duplicated()  # Returns a boolean Series indicating duplicate rows
print("Duplicate indicator for each row:")

# Determine the total number of duplicates
total_duplicates = duplicates.sum()  # Sums the boolean Series to count duplicates
print("Total number of duplicates:", total_duplicates)

Duplicate indicator for each row:
Total number of duplicates: 16656


In [21]:
# Step 2: Display duplicate rows
duplicate_rows = energy_remove_duplicate[energy_remove_duplicate.duplicated()]  # Extracts duplicate rows
print("Duplicate rows:")
duplicate_rows.head()

Duplicate rows:


,borough,account_name,serial_number,funding_origin,total_bill,kwh_consumption,kwh_bill,kw_consumption,kw_bill,year_month,start_date,end_date
102353,brooklyn,williamsburg,6484342,federal,2762.92,19360.0,1369.82,76.8,1369.82,2013-11-01,2013-10-23,2013-11-21
102354,brooklyn,williamsburg,6484344,federal,2922.76,20480.0,941.75,52.8,941.75,2013-11-01,2013-10-23,2013-11-21
102355,brooklyn,williamsburg,6484356,federal,5845.60,40960.0,1940.58,108.8,1940.58,2013-11-01,2013-10-23,2013-11-21
102356,brooklyn,williamsburg,6484397,federal,913.32,6400.0,1112.98,62.4,1112.98,2013-11-01,2013-10-23,2013-11-21
102357,brooklyn,williamsburg,6484474,federal,5845.60,40960.0,1483.97,83.2,1483.97,2013-11-01,2013-10-23,2013-11-21


In [22]:
# Save to clean data 
energy_clean = energy_clean_textual_data.copy()
energy_clean.to_csv('../data/processed/energy_clean.csv', index = False)